<a href="https://colab.research.google.com/github/VickkiMars/AI-ML/blob/main/Multiclass_names_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install names-dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.6 MB/s eta 0:00:00
  Created wheel for names-dataset: filename=names_dataset-3.1.0-py3-none-any.whl size=116832758 sha256=882225b187a385111064b5b3fcffb9ae771ada02d1ba8b7aac0ed21232a1bc6a
  Stored in directory: /root/.cache/pip/wheels/cf/f8/43/0c4aba87b34e971e7255a41f11dc0035c5e55b026dc3480986
Successfully built names-dataset


In [66]:
from names_dataset import NameDataset
import unicodedata
import string
import tensorflow as tf
import random
import logging
import pandas as pd
import os
import numpy as np

In [90]:
print(tf.executing_eagerly())

True


In [9]:
logging.basicConfig(level=logging.DEBUG)


In [15]:
all_letters = string.ascii_letters + ".,;'"
n_letters = len(all_letters)
LENGTH = 10
DIR = os.getcwd()

In [5]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn' and c in all_letters
    )

In [12]:
nd = NameDataset()

In [17]:
def get_country_names(
        length:int,
        code:str,
        ) -> list:

    #['GE', 'NG', 'CH', 'IT']
    male_names = nd.get_top_names(3000, country_alpha2=code)[code]['M']
    male_names = [name.lower() for name in male_names]

    female_names = nd.get_top_names(3000, country_alpha2=code)[code]['F']
    female_names = [name.lower() for name in female_names]
    return male_names, female_names

In [7]:
def load_dataset(filename):
    dataset = tf.data.experimental.CsvDataset(
        header=True,
        filenames=filename,
        record_defaults=[
            tf.string
        ]
    )


In [18]:
# Get Chinese names
logging.info('Retrieving Chinese names...')
chinese_male, chinese_female = get_country_names(LENGTH, 'CH')
logging.info('Chinese names Retrieved!')
# Apply ASCII formatting
logging.info('Formatting Chinese Names...')
chinese_male, chinese_female = [unicodeToAscii(name) for name in chinese_male], [unicodeToAscii(name) for name in chinese_female]
logging.info('Chinese Names Formatted')


# Get Nigerian names
logging.info('Retrieving Nigerian Names...')
nigerian_male, nigerian_female = get_country_names(LENGTH, 'NG')
logging.info('Nigerian Names Retrieved')
# Apply ASCII formatting
logging.info('Formatting Nigerian Names...')
nigerian_male, nigerian_female = [unicodeToAscii(name) for name in nigerian_male],[unicodeToAscii(name) for name in nigerian_female]
logging.info('Nigerian Names formatted.')

logging.info('Retrieving Italian Names...')
# Get Italian names
italian_male, italian_female = get_country_names(LENGTH, 'IT')
logging.info('Italian Names Retrieved.')
# Apply ASCII formatting
logging.info('Formatting Italian Names...')
italian_male, italian_female = [unicodeToAscii(name) for name in italian_male], [unicodeToAscii(name) for name in italian_female]
logging.info('Italian Names Formatted.')


# Get German names
logging.info('Retrieving German Names...')
german_male, german_female = get_country_names(LENGTH, 'GE')
logging.info('German Names retrieved')
# Apply ASCII formatting
logging.info('Formatting German Names...')
german_male, german_female = [unicodeToAscii(name) for name in german_male], [unicodeToAscii(name) for name in german_female]
logging.info('German Names Formatted')

logging.info('Shuffling Names...')


In [50]:
male, female = [chinese_male, italian_male, german_male, nigerian_male],[chinese_female, italian_female, german_female, nigerian_female]

In [55]:
def find_min(male, female):
    if len(male) != len(female):
      i, j = len(male), len(female)
      min_ = min(i,j)
      male, female = male[:min_], female[:min_]
    return male,female

In [56]:
chinese_male, chinese_female = find_min(chinese_male, chinese_female)
italian_male, italian_female = find_min(italian_male, italian_female)
german_male, german_female = find_min(german_male, german_female)
nigerian_male, nigerian_female = find_min(nigerian_male, nigerian_female)

In [57]:
def shuffle_(array:list):
  random.shuffle(array)

In [58]:
chinese = chinese_male + chinese_female
italian = italian_male + italian_female
german = german_male + german_female
nigerian = nigerian_male + nigerian_female

shuffle_(chinese)
shuffle_(italian)
shuffle_(german)
shuffle_(nigerian)
logging.info('Names Shuffled')

In [62]:
min_ = min(len(chinese), len(italian), len(german), len(nigerian))
chinese, italian, german, nigerian = chinese[:min_], italian[:min_], german[:min_], nigerian[:min_]

In [130]:
ALL = {'Chinese':chinese,
       'Italian':italian,
       'German':german,
       'Nigerian':nigerian
       }

In [131]:
logging.info('Loading into .csv format...')
df = pd.DataFrame(ALL)
name, dir = "names_dataset.csv", DIR
df.to_csv(name)
logging.info(f'File saved as {name} at {DIR}')

In [132]:
def send_all(download=False):
    afri = ALL
    return afri

In [133]:
df = load_dataset(DIR+"/"+name)

In [134]:
dataset = send_all()
labels=  dataset.keys()
#print(dataset['Italian'][:5])

lang = [dataset[i] for i in labels]
lang_ = [value for value in dataset.values()][0]
#print(lang_[:10])
lang_ = [len(i) for i in lang_]
VOCAB_SIZE = max(lang_)
#print(max_)

In [135]:
# Combine all text data and create labels
all_text = []
labels = []
for lang, sentences in dataset.items():
    all_text.extend(sentences)
    labels.extend([lang] * len(sentences))

In [136]:
# Shuffle data and labels in the same order
combined = list(zip(all_text, labels))
random.shuffle(combined)
all_text, labels = zip(*combined)

In [137]:
# Split into training, validation, and testing sets (adjust ratios as needed)
train_split = int(0.8 * len(all_text))
val_split = int(0.9 * len(all_text))

train_text = all_text[:train_split]
train_labels = labels[:train_split]
val_text = all_text[train_split:val_split]
val_labels = labels[train_split:val_split]
test_text = all_text[val_split:]
test_labels = labels[val_split:]

In [138]:
# Tokenization and Padding (using TextVectorization)
VOCAB_SIZE = 10000  # Adjust as needed
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_text)

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

In [139]:
# Convert text to numerical sequences
train_sequences = encoder(train_text)
val_sequences = encoder(val_text)
test_sequences = encoder(test_text)

In [156]:
# Pad sequences to a uniform length (find maximum length first)
max_length = max([len(seq) for seq in train_sequences])
train_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length)
val_sequences = tf.keras.preprocessing.sequence.pad_sequences(val_sequences, maxlen=max_length)
test_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length)

# # Reshape the input data for the Conv1D layer
# train_sequences = train_sequences.reshape(train_sequences.shape[0], train_sequences.shape[1], 1)
# val_sequences = val_sequences.reshape(val_sequences.shape[0], val_sequences.shape[1], 1)
# test_sequences = test_sequences.reshape(test_sequences.shape[0], test_sequences.shape[1], 1)

In [157]:
num_classes = len(dataset)  # Number of languages (classes)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [158]:
# ... (Model building steps from previous response)

# 3. Model Training

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Prepare labels for training (one-hot encoding)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

from tensorflow.keras.utils import to_categorical
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=num_classes)
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=num_classes)


In [159]:
# Train the model, capturing the training history
history = model.fit(train_sequences, train_labels_onehot,
          epochs=10,  # Adjust the number of epochs
          validation_data=(val_sequences, val_labels_onehot))

# Check if history is None
if history is None:
    print("Warning: Model training returned None. Check for early stopping or callback issues.")
else:
    # Print a summary of the training results
    print(history.history)

Epoch 1/10


ValueError: Kernel shape must have the same length as input, but received kernel of shape (5, 64, 64) and input of shape (32, 1, 1, 64).

In [155]:
train_sequences

array([[[   1]],

       [[   1]],

       [[8139]],

       ...,

       [[ 684]],

       [[1699]],

       [[8341]]], dtype=int32)